In [1]:
from numpy import *
import operator

def createDataSet():
    group = array([[1.0,1.1], [1.0,1.0], [0,0], [0,0.1]])
    labels = ['A', 'A', 'B', 'B']
    return group, labels

In [2]:
group, labels = createDataSet()

In [3]:
group

array([[1. , 1.1],
       [1. , 1. ],
       [0. , 0. ],
       [0. , 0.1]])

In [4]:
labels

['A', 'A', 'B', 'B']

In [5]:
def classify0(inX, dataset, labels, k):
    dataSize = dataset.shape[0]
    #计算欧式距离
    diffMat = tile(inX, (dataSize, 1)) - dataset#tile()复制向量
    sqdiffMat = diffMat**2
    sqDistances = sqdiffMat.sum(axis=1)
    distances = sqDistances**0.5
    sortDistances = distances.argsort()#argsort()返回数组值从小到大的索引值
    #选择距离最小的k个点
    classCount = {}
    for i in range(k):
        voteIlabel = labels[sortDistances[i]]
        classCount[voteIlabel] = classCount.get(voteIlabel,0) + 1
    #classCount = list(zip(classCount.keys(), classCount.values()))
    sortedClassCount = sorted(classCount.items(), key=operator.itemgetter(1), reverse=True)
    #原书中用的iteritems()在python3.X版本中已被移除
    return sortedClassCount[0][0]

In [6]:
inX = [0.5,1.0]
k = 3
classify0(inX, group, labels, k)

'A'

## 海伦收集
## 样本包含三个特征
- 每年获得的飞行常客里程数
- 玩视频游戏所耗时间百分比
- 每周消费的冰淇淋公升数

In [7]:
#对文件进行处理
def file2matrix(filename):
    fr = open(filename)#打开文件
    arrayOlines = fr.readlines()#读取数据
    numberOflines = len(arrayOlines)#得到文件行数
    returnMat = zeros((numberOflines, 3))
    classLabel = []
    index = 0
    for line in arrayOlines:
        line = line.strip()
        listfromlines = line.split('\t')
        returnMat[index, :] = listfromlines[0:3]
        classLabel.append(int(listfromlines[-1]))
        index += 1
    return returnMat,classLabel

In [8]:
X,Y = file2matrix('datingTestSet2.txt')

In [9]:
X,Y

(array([[4.0920000e+04, 8.3269760e+00, 9.5395200e-01],
        [1.4488000e+04, 7.1534690e+00, 1.6739040e+00],
        [2.6052000e+04, 1.4418710e+00, 8.0512400e-01],
        ...,
        [2.6575000e+04, 1.0650102e+01, 8.6662700e-01],
        [4.8111000e+04, 9.1345280e+00, 7.2804500e-01],
        [4.3757000e+04, 7.8826010e+00, 1.3324460e+00]]),
 [3,
  2,
  1,
  1,
  1,
  1,
  3,
  3,
  1,
  3,
  1,
  1,
  2,
  1,
  1,
  1,
  1,
  1,
  2,
  3,
  2,
  1,
  2,
  3,
  2,
  3,
  2,
  3,
  2,
  1,
  3,
  1,
  3,
  1,
  2,
  1,
  1,
  2,
  3,
  3,
  1,
  2,
  3,
  3,
  3,
  1,
  1,
  1,
  1,
  2,
  2,
  1,
  3,
  2,
  2,
  2,
  2,
  3,
  1,
  2,
  1,
  2,
  2,
  2,
  2,
  2,
  3,
  2,
  3,
  1,
  2,
  3,
  2,
  2,
  1,
  3,
  1,
  1,
  3,
  3,
  1,
  2,
  3,
  1,
  3,
  1,
  2,
  2,
  1,
  1,
  3,
  3,
  1,
  2,
  1,
  3,
  3,
  2,
  1,
  1,
  3,
  1,
  2,
  3,
  3,
  2,
  3,
  3,
  1,
  2,
  3,
  2,
  1,
  3,
  1,
  2,
  1,
  1,
  2,
  3,
  2,
  3,
  2,
  3,
  2,
  1,
  3,
  3,
  3,
  1,
  3,


In [10]:
#使用Matplotlib创建散点图
import matplotlib
import matplotlib.pyplot as plt
fig = plt.figure()
ax = fig.add_subplot(111)
#ax.scatter(X[:,1], X[:,2])
ax.scatter(X[:,1],X[:,2], 15.0*array(Y), 15.0*array(Y))
plt.show()

<Figure size 640x480 with 1 Axes>

In [11]:
#数据归一化
def autoNorm(dataSet):
    minVals = dataSet.min(0)
    maxVals = dataSet.max(0)
    ranges = maxVals - minVals
    normDataset = zeros(shape(dataSet))
    m = dataSet.shape[0]
    cpmin = tile(minVals, (m, 1))
    normDataset = dataSet - cpmin
    cprange = tile(ranges, (m, 1))
    normDataset = normDataset/cprange
    return normDataset, ranges, minVals

In [12]:
normDataset, ranges, minVals = autoNorm(X)
normDataset

array([[0.44832535, 0.39805139, 0.56233353],
       [0.15873259, 0.34195467, 0.98724416],
       [0.28542943, 0.06892523, 0.47449629],
       ...,
       [0.29115949, 0.50910294, 0.51079493],
       [0.52711097, 0.43665451, 0.4290048 ],
       [0.47940793, 0.3768091 , 0.78571804]])

In [18]:
#测试样本
def datingClassTest():
    hoTatio = 0.10
    datingDataMat, datingLabels = file2matrix('datingTestSet2.txt')#获取数据
    normMat, ranges, minVals = autoNorm(datingDataMat)#数据归一化
    m = normMat.shape[0] #测试样本个数
    numTestVecs = int(m * hoTatio)
    errorCount = 0.0
    for i in range(numTestVecs):
        classifierResult = classify0(normMat[i,:], normMat[numTestVecs:m,:], datingLabels[numTestVecs:m], 3)
       #print("the classifier came back with: %d, the real answer is: %d" % (classifierResult, datingLabels[i]))
        if(classifierResult != datingLabels[i]): errorCount += 1.0
    print("the total error rate is: %f" % (errorCount/float(numTestVecs)))

In [19]:
datingClassTest()

the total error rate is: 0.050000


In [23]:
#构建完整的可用系统 约会网站预测函数
def classifyPerson():
    resultList = ['not at all', 'in small doses', 'in large doses']
    #输入测试数据
    percentTats = float(input("percentage of time spent playing video game?"))
    ffMiles = float(input("frequent flier miles earned per year?"))
    iceCream = float(input("liters of ice cream consumed per year?"))
    #读取文件数据
    datingDataMat, datingLabels = file2matrix('datingTestSet2.txt')
    #归一化数据
    normMat, ranges, minVals = autoNorm(datingDataMat)
    inArr = array([ffMiles, percentTats, iceCream])
    classifierResult = classify0((inArr-minVals)/ranges, normMat, datingLabels, 3)
    print("You will probably like this person: ",resultList[classifierResult - 1])

In [25]:
classifyPerson()

percentage of time spent playing video game?10
frequent flier miles earned per year?10000
liters of ice cream consumed per year?0.5
You will probably like this person:  in small doses


## 手写识别系统
- 收集数据： 提供文本文件
- 准备数据： 编写函数classify0（）， 将图像格式转换为分类器使用的list格式
- 分析数据： 在python命令提示符中检查数据， 确保它符合要求
- 训练算法： 此步骤不适合用于k-近邻算法
- 测试算法： 编写函数使用提供的部分数据集作为测试样本， 测试样本与费测试样本的区别在于测试样本是已经完成分类的数据， 如果预测分类与实际类别不同， 则标记为一个错误
- 使用算法： 本例没有完成此步骤， 若你感兴趣可以构建完整的应用程序， 从图像中提取数字， 并完成数字识别，美国的邮件分拣系统就是一个实际运行的类似系统

In [29]:
#图像转向量函数
def img2vector(filename):
    returnVect = zeros((1,1024))
    fr = open(filename)
    for i in range(32):
        lineStr = fr.readline()
        for j in range(32):
            returnVect[0, 32*i + j] = int(lineStr[j])
    return returnVect

testVector = img2vector('testDigits/0_13.txt')
testVector[0, 0:31]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [33]:
from os import listdir #从os模块中导入listdir, 它可以列出给定目录的文件名

#手写数字识别系统测试代码
def handwritingClassTest():
    hwLabels = []
    #获取文件目录
    trainingFileList = listdir('trainingDigits')
    m = len(trainingFileList)
    trainingMat = zeros((m, 1024))
    #从文件名解析分类数字
    for i in range(m):
        fileNameStr = trainingFileList[i]
        fileStr = fileNameStr.split('.')[0]
        classNumStr = int(fileStr.split('_')[0])
        hwLabels.append(classNumStr)
        trainingMat[i, :] = img2vector('trainingDigits/%s' % fileNameStr)
    testFileList = listdir('testDigits')
    errorCount = 0.0
    mTest = len(testFileList)
    #读取测试文件进行测试
    for i in range(mTest):
        fileNameStr = testFileList[i]
        fileStr = fileNameStr.split('.')[0]
        classNumStr = int(fileNameStr.split('_')[0])
        vectorUnderTest = img2vector('testDigits/%s' % fileNameStr)
        classifierResult = classify0(vectorUnderTest, trainingMat, hwLabels, 3)
        print("the classifier came back with: %d, the real answer is: %d" % (classifierResult, classNumStr))
        if(classifierResult != classNumStr): errorCount += 1.0
    print("\nthe total number of errors is: %d" % errorCount)
    print("\nthe total error rate is: %f" % (errorCount/float(mTest)))
        
handwritingClassTest()

the classifier came back with: 9, the real answer is: 9
the classifier came back with: 9, the real answer is: 9
the classifier came back with: 2, the real answer is: 2
the classifier came back with: 0, the real answer is: 0
the classifier came back with: 4, the real answer is: 4
the classifier came back with: 7, the real answer is: 7
the classifier came back with: 2, the real answer is: 2
the classifier came back with: 4, the real answer is: 4
the classifier came back with: 8, the real answer is: 8
the classifier came back with: 6, the real answer is: 6
the classifier came back with: 2, the real answer is: 2
the classifier came back with: 5, the real answer is: 5
the classifier came back with: 4, the real answer is: 4
the classifier came back with: 0, the real answer is: 0
the classifier came back with: 8, the real answer is: 8
the classifier came back with: 5, the real answer is: 5
the classifier came back with: 3, the real answer is: 3
the classifier came back with: 5, the real answe

the classifier came back with: 5, the real answer is: 5
the classifier came back with: 5, the real answer is: 5
the classifier came back with: 2, the real answer is: 2
the classifier came back with: 3, the real answer is: 3
the classifier came back with: 8, the real answer is: 8
the classifier came back with: 9, the real answer is: 9
the classifier came back with: 4, the real answer is: 4
the classifier came back with: 7, the real answer is: 7
the classifier came back with: 5, the real answer is: 5
the classifier came back with: 2, the real answer is: 2
the classifier came back with: 2, the real answer is: 2
the classifier came back with: 7, the real answer is: 7
the classifier came back with: 6, the real answer is: 6
the classifier came back with: 7, the real answer is: 1
the classifier came back with: 7, the real answer is: 7
the classifier came back with: 1, the real answer is: 1
the classifier came back with: 0, the real answer is: 0
the classifier came back with: 6, the real answe

the classifier came back with: 3, the real answer is: 3
the classifier came back with: 0, the real answer is: 0
the classifier came back with: 3, the real answer is: 3
the classifier came back with: 1, the real answer is: 1
the classifier came back with: 1, the real answer is: 1
the classifier came back with: 7, the real answer is: 7
the classifier came back with: 3, the real answer is: 3
the classifier came back with: 7, the real answer is: 7
the classifier came back with: 1, the real answer is: 1
the classifier came back with: 9, the real answer is: 9
the classifier came back with: 0, the real answer is: 0
the classifier came back with: 6, the real answer is: 6
the classifier came back with: 9, the real answer is: 9
the classifier came back with: 0, the real answer is: 0
the classifier came back with: 7, the real answer is: 7
the classifier came back with: 7, the real answer is: 7
the classifier came back with: 9, the real answer is: 9
the classifier came back with: 1, the real answe

the classifier came back with: 7, the real answer is: 7
the classifier came back with: 4, the real answer is: 4
the classifier came back with: 2, the real answer is: 2
the classifier came back with: 1, the real answer is: 8
the classifier came back with: 6, the real answer is: 6
the classifier came back with: 4, the real answer is: 4
the classifier came back with: 7, the real answer is: 7
the classifier came back with: 2, the real answer is: 2
the classifier came back with: 1, the real answer is: 1
the classifier came back with: 4, the real answer is: 4
the classifier came back with: 8, the real answer is: 8
the classifier came back with: 7, the real answer is: 7
the classifier came back with: 0, the real answer is: 0
the classifier came back with: 8, the real answer is: 8
the classifier came back with: 9, the real answer is: 9
the classifier came back with: 2, the real answer is: 2
the classifier came back with: 9, the real answer is: 9
the classifier came back with: 9, the real answe

the classifier came back with: 6, the real answer is: 6
the classifier came back with: 4, the real answer is: 4
the classifier came back with: 0, the real answer is: 0
the classifier came back with: 7, the real answer is: 7
the classifier came back with: 2, the real answer is: 2
the classifier came back with: 9, the real answer is: 9
the classifier came back with: 8, the real answer is: 8
the classifier came back with: 8, the real answer is: 8
the classifier came back with: 4, the real answer is: 4
the classifier came back with: 6, the real answer is: 6
the classifier came back with: 5, the real answer is: 5
the classifier came back with: 6, the real answer is: 6
the classifier came back with: 8, the real answer is: 8
the classifier came back with: 7, the real answer is: 7
the classifier came back with: 7, the real answer is: 7
the classifier came back with: 6, the real answer is: 6
the classifier came back with: 9, the real answer is: 9
the classifier came back with: 0, the real answe

the classifier came back with: 1, the real answer is: 1
the classifier came back with: 9, the real answer is: 9
the classifier came back with: 5, the real answer is: 5
the classifier came back with: 9, the real answer is: 9
the classifier came back with: 6, the real answer is: 6
the classifier came back with: 4, the real answer is: 4
the classifier came back with: 1, the real answer is: 1
the classifier came back with: 1, the real answer is: 1
the classifier came back with: 4, the real answer is: 4
the classifier came back with: 3, the real answer is: 3
the classifier came back with: 6, the real answer is: 6
the classifier came back with: 8, the real answer is: 8
the classifier came back with: 4, the real answer is: 4
the classifier came back with: 0, the real answer is: 0
the classifier came back with: 4, the real answer is: 4
the classifier came back with: 7, the real answer is: 7
the classifier came back with: 6, the real answer is: 6
the classifier came back with: 8, the real answe

the classifier came back with: 0, the real answer is: 0
the classifier came back with: 8, the real answer is: 8

the total number of errors is: 12

the total error rate is: 0.012685
